In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install transformers[torch]
!pip install --upgrade transformers accelerate

In [4]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments,T5TokenizerFast
import torch

In [5]:
# Set the model name
model_name = "csebuetnlp/mT5_multilingual_XLSum"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:
!pip install pandas
!pip install xmltodict
!pip install gdown

import pandas as pd
import xmltodict
import gdown


In [9]:
# Read the XML file and convert to DataFrame
output = "/content/drive/MyDrive/Copy of EGY_alahram_news.xml"

with open(output, 'r') as xml_file:
    xml_content = xml_file.read()

data = xmltodict.parse(xml_content)
df = pd.json_normalize(data)  # Convert XML to DataFrame

# Display the DataFrame
print(df)

                                          items.item
0  [{'address': 'http://gate.ahram.org.eg/News/23...


In [10]:
  df['items.item'][0]

[{'address': 'http://gate.ahram.org.eg/News/2348461.aspx',
  'title': 'جهاز مدينة السادات يشن حملة لإزالة الإعلانات المخالفة صور',
  'article': 'وقال المهندس أمين غنيم إن الحملة أسفرت عن إزالة عدة إعلانات مخالفة وغير مرخصة، وتم اتخاذ الإجراءات القانونية ضد المخالفين، مؤكداً استمرار عمل اللجنة المشكلة لحين التأكد من إزالة جميع الإعلانات المخالفة بنطاق المدينة. وشدد رئيس الجهاز على الإدارات المعنية التصدي لتلك الظاهرة، وإزالة جميع الإعلانات المخالفة وغير المرخصة بجميع أنحاء المدينة، موجهاً بتطبيق القانون على الجميع دون أي استثناءات حفاظاً على المظهر الحضاري لمدينة السادات. وفي سياق متصل، أشار المهندس أمين غنيم، إلى أن إدارة التشغيل والصيانة بجهاز المدينة قامت بالاستجابة السريعة لتغطية ٩ بالوعات صرف بالمنطقة السكنية العاشرة بالمدينة، بعد إبلاغ المواطنين عن سرقة أغطيتها من قبل مجهولين. وقال المهندس أمين غنيم يهيب جهاز مدينة السادات بالمواطنين الإبلاغ الفوري عن أي غرفة صرف بدون غطاء، أو عن أي شخصٍ يحاول سرقة الأغطية على الأرقام الخط الساخن ١٥١٠٠ إدارة التشغيل والصيانة ٠٤٨٢٦٠١٧٢١ .',
  'summ

In [11]:
# Normalize the 'items.item' column into separate columns
df_normalized = pd.json_normalize(df['items.item'][0])

df_normalized

,address,title,article,summary
0,http://gate.ahram.org.eg/News/2348461.aspx,جهاز مدينة السادات يشن حملة لإزالة الإعلانات ا...,وقال المهندس أمين غنيم إن الحملة أسفرت عن إزال...,صرح المهندس أمين غنيم، رئيس جهاز تنمية مدينة ا...
1,http://gate.ahram.org.eg/News/2348471.aspx,مستشار مفتي الجمهورية يشارك في أسبوع التعريف ب...,وتأتي هذه المشاركة انطلاقًا من حرص دار الإفتاء...,توجه الدكتور إبراهيم نجم مستشار مفتي الجمهورية...
2,http://gate.ahram.org.eg/News/2348457.aspx,جامعة حلوان تنظم دورة تدريب المدربين للمستوى ا...,تهدف الدورة التدريبية إلى التدريب على أحدث ما ...,تنظم وحدة برامج التنمية البشرية بالإدارة العام...
3,http://gate.ahram.org.eg/News/2348454.aspx,وزير الأوقاف يفتتح تطوير وحدة القسطرة بمستشفى ...,يأتي ذلك في إطار حرص وزارة الأوقاف على تقديم خ...,يفتتح وزير الأوقاف الدكتور محمد مختار جمعة، وح...
4,http://gate.ahram.org.eg/News/2348470.aspx,طلب إحاطة لرئيس الوزراء ووزيري الأوقاف والتنمي...,وقالت شيرين القشاش إنه منذ صدور هذا الحكم وحتى...,وجهت الدكتورة شيرين القشاش عضو مجلس النواب طلب...
...,...,...,...,...
315892,http://gate.ahram.org.eg/News/2348473.aspx,وزيرا الإسكان والشباب يبحثان توفيرعضويات بأسعا...,وأوضح وزير الإسكان، أن ذلك يأتي اتساقاً مع أهم...,عقد الدكتور عاصم الجزار، وزير الإسكان والمرافق...
315893,http://gate.ahram.org.eg/News/2348473.aspx,وزيرا الإسكان والشباب يبحثان توفيرعضويات بأسعا...,وأوضح وزير الإسكان، أن ذلك يأتي اتساقاً مع أهم...,عقد الدكتور عاصم الجزار، وزير الإسكان والمرافق...
315894,http://gate.ahram.org.eg/News/2348473.aspx,وزيرا الإسكان والشباب يبحثان توفيرعضويات بأسعا...,وأوضح وزير الإسكان، أن ذلك يأتي اتساقاً مع أهم...,عقد الدكتور عاصم الجزار، وزير الإسكان والمرافق...
315895,http://gate.ahram.org.eg/News/2348473.aspx,وزيرا الإسكان والشباب يبحثان توفيرعضويات بأسعا...,وأوضح وزير الإسكان، أن ذلك يأتي اتساقاً مع أهم...,عقد الدكتور عاصم الجزار، وزير الإسكان والمرافق...


In [12]:
# Drop the 'address' and 'title' columns
df_normalized = df_normalized.drop(['address', 'title'], axis=1)

df_normalized.head()

,article,summary
0,وقال المهندس أمين غنيم إن الحملة أسفرت عن إزال...,صرح المهندس أمين غنيم، رئيس جهاز تنمية مدينة ا...
1,وتأتي هذه المشاركة انطلاقًا من حرص دار الإفتاء...,توجه الدكتور إبراهيم نجم مستشار مفتي الجمهورية...
2,تهدف الدورة التدريبية إلى التدريب على أحدث ما ...,تنظم وحدة برامج التنمية البشرية بالإدارة العام...
3,يأتي ذلك في إطار حرص وزارة الأوقاف على تقديم خ...,يفتتح وزير الأوقاف الدكتور محمد مختار جمعة، وح...
4,وقالت شيرين القشاش إنه منذ صدور هذا الحكم وحتى...,وجهت الدكتورة شيرين القشاش عضو مجلس النواب طلب...


In [18]:
!pip install qalsadi

In [19]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import qalsadi.lemmatizer

nltk.download('stopwords')
nltk.download('punkt')

# Create a Qalsadi lemmatizer object
lemmatizer = qalsadi.lemmatizer.Lemmatizer()

# Set up stop words
stop_words = set(stopwords.words('arabic'))

# Define a function to apply the preprocessing steps to a text
def preprocess_text(text):

    # Tokenize
    tokenized_text = word_tokenize(text)  # Replace with your preferred tokenizer if needed

    # Remove stop words
    stop_words_removed = [word for word in tokenized_text if word not in stop_words]

    # Lemmatize
    lemmatized_text = [lemmatizer.lemmatize(word) for word in stop_words_removed]

    # Join the lemmatized words back into a string
    preprocessed_text = ' '.join(lemmatized_text)

    return preprocessed_text

# Apply the preprocessing function to the 'article' and 'summary' columns in the DataFrame
df_normalized['article'] = df_normalized['article'].apply(preprocess_text)
# df_normalized['summary'] = df_normalized['summary'].apply(preprocess_text)

# Display the updated DataFrame
df_normalized


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,article,summary
0,تعقد الجلسة برئاسة المستشار محمد عامر جادو، وع...,تنظر غدا محكمة جنايات القاهرة، التظلم المقدم م...
1,قرر المحافظ تفعيل خطوط الإنتاج بالمصنع ومشاركة...,أعلن محمد الزملوط محافظ الوادى الجديد، عن تخصي...
2,وقالت الوكالة الفلسطينية الرسمية اليوم الأحد، ...,أفادت وكالة الأنباء والمعلومات الفلسطينية وفا ...
3,اختصمت الدعوى حملت رقم ٧٢٤٢١ لسنة ٧١ قضائية وز...,تستكمل الدائرة الثانية بمحكمة القضاء الإداري ب...
4,وقد خضع جنينه النيابة العسكرية، كون ذكره يتعلق...,قررت النيابة العسكرية حبس رئيس الجهاز المركزي ...
...,...,...
5495,نشوب حريق داخل مخازن الغلال بجمرك بورسعيد,اندلع منذ قليل حريق هائل بمخازن الغلال بجمارك ...
5496,أثناء مرور قوة أمنية مباحث إدارة تامين الطرق و...,تمكنت مباحث القاهرة من القبض على ٣ عاطلين وبحو...
5497,وأضاف عبدالغفار، كلمته خلال توقيع بروتوكول الت...,قال الدكتور خالد عبدالغفار، وزير التعليم العال...
5498,يعرض فيلم انتزاع الكهرمان للمخرج حسين شريف، وا...,يعرض مهرجان الأقصر للسينما الإفريقية، فى دورته...


In [20]:
pip install huggingface_hub

In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
train_df = df_normalized.sample(frac=0.9)
train_df.reset_index(drop=True, inplace=True)


In [23]:
valid_df = df_normalized.sample(frac=0.1)
valid_df.reset_index(drop=True, inplace=True)


In [24]:
# Set the training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/mt5-ar", num_train_epochs=15, warmup_ratio=0.03,
    per_device_train_batch_size=4,
    save_strategy="epoch",
    save_steps=2,
    logging_steps=100,  push_to_hub=True,
    evaluation_strategy='steps',
    gradient_accumulation_steps=16,
    gradient_checkpointing=True, hub_private_repo=True,
    overwrite_output_dir=True
)

# Define the PyTorch dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.input_ids = df['article'].apply(lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=1024))
        self.target_ids = df['summary'].apply(lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=256))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx]).to(device),
            'attention_mask': torch.tensor([1] * len(self.input_ids[idx])).to(device),
            'labels': torch.tensor(self.target_ids[idx]).to(device),
            'decoder_attention_mask': torch.tensor([1] * len(self.target_ids[idx])).to(device)
        }

# Create the dataset
train_dataset = CustomDataset(train_df)
eval_dataset = CustomDataset(valid_df)

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset = eval_dataset
)

# Train the model
trainer.train()

Cloning https://huggingface.co/MostafaKhidr/mt5-ar into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
